In [1]:
import pandas as pd
from pprint import pprint
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument, PDFNoOutlines
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTPage, LTChar, LTAnno, LAParams, LTTextBox, LTTextLine

class PDFPageDetailedAggregator(PDFPageAggregator):
    def __init__(self, rsrcmgr, pageno=1, laparams=None):
        PDFPageAggregator.__init__(self, rsrcmgr, pageno=pageno, laparams=laparams)
        self.rows = []
        self.page_number = 0
    def receive_layout(self, ltpage):        
        def render(item, page_number):
            if isinstance(item, LTPage) or isinstance(item, LTTextBox):
                for child in item:
                    render(child, page_number)
            elif isinstance(item, LTTextLine):
                child_str = ''
                for child in item:
                    if isinstance(child, (LTChar, LTAnno)):
                        child_str += child.get_text()
                ##child_str = ' '.join(child_str.split()).strip() LO TOLGO PER QUESTO DATASET
                if child_str:
                    row = (page_number, item.bbox[0], item.bbox[1], item.bbox[2], item.bbox[3], child_str) # bbox == (x1, y1, x2, y2)
                    self.rows.append(row)
                for child in item:
                    render(child, page_number)
            return
        render(ltpage, self.page_number)
        self.page_number += 1
        self.rows = sorted(self.rows, key = lambda x: (x[0], -x[2]))
        self.result = ltpage

fp = open("Goethe-Zertifikat_A2_Wortliste.pdf", 'rb')
parser = PDFParser(fp)
doc = PDFDocument(parser)
#doc.initialize() # leave empty for no password

rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageDetailedAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

for page in PDFPage.create_pages(doc):
    interpreter.process_page(page)
    # receive the LTPage object for this page
    device.get_result()

In [2]:
DF = pd.DataFrame(device.rows)
DF.columns = ["page","xmin","ymin","xmax","ymax","text"]
alphw = DF[(DF["page"]>7)&(DF["page"]<31)&(DF["ymin"]<806)&(DF["ymin"]>27)&(DF["xmin"]>30)]

In [3]:
def columntranslation(x):
    if ((x["xmin"]>30) & (x["xmin"]<40)):
        if (x["xmax"]<110):
            return [1]
        return [1,2]
    if ((x["xmin"]>100) & (x["xmin"]<110)):
        return [2]
    if ((x["xmin"]>300) & (x["xmin"]<310)):
        if (x["xmax"]<380):
            return [3]
        return [3,4]
    if ((x["xmin"]>370) & (x["xmin"]<380)):
        return [4]
    return [0]


In [4]:
alphw["columns"] = alphw.apply(columntranslation,axis=1)

<ipython-input-4-586d9643806c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alphw["columns"] = alphw.apply(columntranslation,axis=1)


In [5]:
import re
alphw["texts"]=alphw["text"].apply(lambda x: list(map(lambda y: y.strip(), [""]*(x[:2]=="  ")+[z for z in re.split("\s{2,}",x) if z])))

<ipython-input-5-69093e541a72>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alphw["texts"]=alphw["text"].apply(lambda x: list(map(lambda y: y.strip(), [""]*(x[:2]=="  ")+[z for z in re.split("\s{2,}",x) if z])))


In [6]:
alphw["n_columns"] = alphw["columns"].apply(len)
alphw["n_texts"] = alphw["texts"].apply(len)

<ipython-input-6-4387d560560f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alphw["n_columns"] = alphw["columns"].apply(len)
<ipython-input-6-4387d560560f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alphw["n_texts"] = alphw["texts"].apply(len)


In [7]:
alphw.groupby("n_columns")["n_texts"].value_counts()

n_columns  n_texts
1          1           393
2          2          1977
           1            28
Name: n_texts, dtype: int64

In [8]:
alphw["columns"].apply(str).value_counts()

[1, 2]    1007
[3, 4]     998
[1]        169
[3]        164
[4]         30
[2]         30
Name: columns, dtype: int64

In [9]:
correct = alphw[alphw["n_columns"]!=alphw["n_texts"]]
correct.index

Int64Index([ 748, 1108, 1117, 1311, 1351, 1478, 1844, 1872, 1916, 1938, 2155,
            2165, 2260, 2263, 2724, 2939, 3019, 3118, 3139, 3140, 3152, 3166,
            3179, 3280, 3379, 3435, 3499, 3525],
           dtype='int64')

In [10]:
alphw.at[748,"texts"]=['die Ausstellung,-en','Gehen wir morgen zusammen in die Ausstellung?']
alphw.at[1108,"texts"]=['dafür/dagegen sein,','Ich bin dafür/dagegen.']
alphw.at[1117,"texts"]=['das Doppelzimmer, -','Möchten Sie ein Doppelzimmer oder ein']
alphw.at[1311,"texts"]=['das Erdgeschoss,-e', 'Ich wohne im Erdgeschoss.']
alphw.at[1351,"texts"]=['die Ermäßigung,-en', 'Für Schüler, Studenten und Rentner gibt es eine']
alphw.at[1478,"texts"]=['der Fotoapparat, -e', 'Ich möchte mir einen Fotoapparat kaufen.']
alphw.at[1844,"texts"]=['die Information,-en', 'Wir haben hier einige wichtige Informationen']
alphw.at[1872,"texts"]=['interessieren (sich),', 'Ich interessiere mich für Fußball.']
alphw.at[1916,"texts"]=['die Kenntnisse (Pl.)','Deine Deutschkenntnisse sind sehr gut!']
alphw.at[1938,"texts"]=['der Kindergarten,¨-','Die kleine Laura geht schon in den Kindergarten.']
alphw.at[2155,"texts"]=['die Mannschaft,-en','Meine Lieblingsmannschaft hat 1:0 verloren.']

alphw.at[2165,"texts"]=['das Medikament,-e','Nehmen Sie dieses Medikament dreimal pro Tag!']
alphw.at[2260,"texts"]=['-kommen/-machen/', 'Fahren wir mit dem Auto oder mit dem Zug nach']
alphw.at[2263,"texts"]=['die Nachbarin,-nen', 'Nachbarin ist Italienerin.']
alphw.at[2724,"texts"]=['das Schlafzimmer, -','Hier ist das Schlafzimmer.']
alphw.at[2939,"columns"]=[1]
alphw.at[3019,"texts"]=['der Supermarkt, ¨-e','Ich kaufe oft im Supermarkt ein.']
alphw.at[3118,"texts"]=['die Universität, -en','Er studiert an der Universität Freiburg.']
alphw.at[3139,"texts"]=['der Unterricht (Sg.)','Der Unterricht dauert bis 19 Uhr.']
alphw.at[3140,"texts"]=['der Unterschied, -e','Was ist der Unterschied zwischen den beiden']
alphw.at[3152,"texts"]=['die Unterschrift,-en','Hier fehlt noch Ihre Unterschrift.']
alphw.at[3166,"columns"]=[3]

alphw.at[3179,"texts"]=['das Medikament,-e','Nehmen Sie dieses Medikament dreimal pro Tag!']
alphw.at[3280,"texts"]=['-kommen/-machen/', 'Fahren wir mit dem Auto oder mit dem Zug nach']
alphw.at[3379,"texts"]=['die Nachbarin,-nen', 'Nachbarin ist Italienerin.']
alphw.at[3435,"texts"]=['das Wiedersehen, -','Auf Wiedersehen!']
alphw.at[3499,"texts"]=['das Wohnzimmer, -','Die Kinder sitzen im Wohnzimmer und sehen fern.']
alphw.at[3525,"columns"]=[1]

In [11]:
alphw["n_columns"] = alphw["columns"].apply(len)
alphw["n_texts"] = alphw["texts"].apply(len)

<ipython-input-11-4387d560560f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alphw["n_columns"] = alphw["columns"].apply(len)
<ipython-input-11-4387d560560f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alphw["n_texts"] = alphw["texts"].apply(len)


In [12]:
alphw.groupby("n_columns")["n_texts"].value_counts()

n_columns  n_texts
1          1           396
2          2          2002
Name: n_texts, dtype: int64

In [13]:
alphwb=alphw.set_index(["page","xmin","ymin","xmax","ymax","text","n_columns","n_texts"])[["texts","columns"]].apply(pd.Series.explode).reset_index()
alphwb["columnaux"]=1-alphwb["columns"].isin([1,3])
alphwb["yabs"] = (850-alphwb["ymax"]+850*(alphwb["columns"]>2.5)+alphwb["page"].apply(int)*2000-16000).apply(lambda x: int(round(x,0)))

In [14]:
def assignblock(x,limits):
    return sum(x>=limits)

def addspace(x):
    output = x.iloc[0]
    for i in range(1,len(x)):
        xi = x.iloc[i]
        if len(output)>=2:
            if (output[-1]=="-") & (output[-2].isalpha()):
                output=output[:-1]+xi
        output+=" "+xi
            
    return output

In [15]:
entries = alphwb[alphwb["columnaux"]==0]
entries = entries.sort_values(["page","yabs"])
entries["dy"] = entries["yabs"].diff()
entries["groupaux"] = (entries["dy"]>13).cumsum()
limits = entries.groupby("groupaux").yabs.min()-4
alphwb["block"] = alphwb["yabs"].apply(lambda x: assignblock(x,limits))
alphwb = alphwb.sort_values(["columnaux","yabs","xmin"])#.reset_index(drop=True)
final = alphwb.groupby(["columnaux","block"]).agg({"texts":addspace}).reset_index(drop=False)
entries  = final[final["columnaux"]==0]
examples = final[final["columnaux"]==1]
ff = pd.merge(entries, examples, on="block")

In [16]:
ff.loc[1112,"texts_x"]

'zurück(fahren, geben, (fahren, geben, gehen, kommen, laufen)  '

In [17]:
entries[entries["texts"]=="das Schlafzimmer, -"]

,columnaux,block,texts
797,0,798,"das Schlafzimmer, -"


In [19]:
pd.options.display.max_rows = 1200

In [21]:
ff

,columnaux_x,block,texts_x,columnaux_y,texts_y
0,0,1,"der Apparat, -e",1,Was machen wir mit deinem alten Apparat?
1,0,2,,1,Ich habe einen neuen Fotoapparat.
2,0,3,"arbeiten, arbeitet,",1,Wo arbeiten Sie?
3,0,4,"hat gearbeitet die Arbeit, -en",1,Ich arbeite als Krankenschwester in einem Kran...
4,0,5,arbeitslos,1,Seit wann ist er schon arbeitslos? Es gibt bei...
5,0,6,"ärgern (sich), ärgert, hat geärgert",1,"Warum ärgerst du dich? - Ich ärgere mich, weil..."
6,0,7,arm,1,"Sie haben nicht viel Geld, sie sind arm."
7,0,8,"der Arm, -e",1,Mein Arm tut weh.
8,0,9,"der Artikel, -",1,"Im Deutschen gibt es drei Artikel: der, die un..."
9,0,10,auch,1,Ich bin auch Spanier. Maria muss auch am Woche...


In [19]:
alphw[alphw["text"]=="das Wiedersehen, - Auf Wiedersehen!\n"]

,page,xmin,ymin,xmax,ymax,text,columns,texts,n_columns,n_texts
3435,29,304.1293,360.7034,437.6397,368.7034,"das Wiedersehen, - Auf Wiedersehen!\n","[3, 4]","[das Schlafzimmer, -, Hier ist das Schlafzimmer.]",2,2


In [18]:
alphw[alphw["texts"].apply(lambda x:"das Schlafzimmer, -" in x)]

,page,xmin,ymin,xmax,ymax,text,columns,texts,n_columns,n_texts
2724,24,34.5552,717.1434,196.9304,725.1434,"das Schlafzimmer, - Hier ist das Schlafzimmer.\n","[1, 2]","[das Schlafzimmer, -, Hier ist das Schlafzimmer.]",2,2
3435,29,304.1293,360.7034,437.6397,368.7034,"das Wiedersehen, - Auf Wiedersehen!\n","[3, 4]","[das Schlafzimmer, -, Hier ist das Schlafzimmer.]",2,2


In [80]:
entries["texts"].value_counts()

das Schlafzimmer, -                           2
günstig                                       1
privat                                        1
telefonieren, telefoniert, hat telefoniert    1
romantisch                                    1
                                             ..
der Erwachsene,-n                             1
die Reihe, -n                                 1
der Rücken, -                                 1
dünn                                          1
                                              1
Name: texts, Length: 1137, dtype: int64

In [77]:
ff

,columnaux_x,block,texts_x,columnaux_y,texts_y
0,0,1,"der Apparat, -e",1,Was machen wir mit deinem alten Apparat?
1,0,2,,1,Ich habe einen neuen Fotoapparat.
2,0,3,"arbeiten, arbeitet,",1,Wo arbeiten Sie?
3,0,4,"hat gearbeitet die Arbeit, -en",1,Ich arbeite als Krankenschwester in einem Kran...
4,0,5,arbeitslos,1,Seit wann ist er schon arbeitslos? Es gibt bei...
...,...,...,...,...,...
1110,0,1134,"zumachen, macht zu, hat zugemacht",1,Machst du bitte das Fenster zu?
1111,0,1135,zurück,1,"Eine Fahrkarte nach Frankfurt und zurück, bitte."
1112,0,1136,"zurück(fahren, geben, (fahren, geben, gehen, k...",1,Fahrt ihr nach der Party zurück nach Hause? Wa...
1113,0,1137,zusammen,1,"Wollen wir zusammen essen gehen? Zahlen, bitte..."


In [70]:
entries[entries["texts"].isin(["ärgert,","arm", "der Arm, -e", "der Artikel, -"])]

,page,xmin,ymin,xmax,ymax,text,n_columns,n_texts,texts,columns,columnaux,yabs,dy
40,8,35.6892,591.5510,176.5076,599.5510,"ärgert, Wetter schlecht ist...",2,2,"ärgert,",1,0,250,11.0
47,8,35.6892,566.4326,243.0820,574.4326,arm Sie haben nicht vie...,2,2,arm,1,0,276,15.0
51,8,35.6892,552.3142,172.0100,560.3142,"der Arm, -e Mein Arm tut weh.\n",2,2,"der Arm, -e",1,0,290,14.0
54,8,35.6892,538.1958,284.4972,546.1958,"der Artikel, - Im Deutschen gibt es ...",2,2,"der Artikel, -",1,0,304,14.0


In [71]:
entries["dy"].value_counts()

11.0     1200
14.0      933
15.0      119
25.0       16
28.0       12
210.0       4
36.0        3
514.0       3
508.0       3
47.0        3
499.0       3
511.0       3
26.0        3
202.0       2
199.0       2
208.0       2
232.0       2
29.0        2
203.0       2
211.0       2
522.0       2
506.0       2
216.0       1
500.0       1
200.0       1
519.0       1
505.0       1
224.0       1
80.0        1
227.0       1
533.0       1
221.0       1
206.0       1
250.0       1
497.0       1
503.0       1
Name: dy, dtype: int64

In [62]:
alphw

,page,xmin,ymin,xmax,ymax,text,n_columns,n_texts,texts,columns,columnaux,yabs
0,8,35.6892,717.1430,263.1628,725.1430,"der Apparat, -e Was machen wir mit dein...",2,2,"der Apparat, -e",1,0,125
1,8,35.6892,717.1430,263.1628,725.1430,"der Apparat, -e Was machen wir mit dein...",2,2,Was machen wir mit deinem alten Apparat?,2,1,125
2,8,304.6969,717.1430,478.8025,725.1430,aus Michele kommt aus ...,2,2,aus,3,0,975
3,8,304.6969,717.1430,478.8025,725.1430,aus Michele kommt aus ...,2,2,Michele kommt aus Brasilien.,4,1,975
4,8,305.7233,706.1430,557.9593,714.1430,Frau Müller geht...,2,2,,3,0,986
...,...,...,...,...,...,...,...,...,...,...,...,...
4395,30,34.5552,142.5394,278.5816,150.5394,(an/aus)gezogen Die Nachbarn sind am Woch...,2,2,Die Nachbarn sind am Wochenende ausgezogen.,2,1,44699
4396,30,34.5552,128.4210,280.6208,136.4210,"das Ziel, -e John läuft sehr schne...",2,2,"das Ziel, -e",1,0,44714
4397,30,34.5552,128.4210,280.6208,136.4210,"das Ziel, -e John läuft sehr schne...",2,2,John läuft sehr schnell. Er war als erster am ...,2,1,44714
4398,30,35.5816,117.4210,202.1736,125.4210,Was ist dein Zie...,2,2,,1,0,44725


In [55]:
# 3179, 3280, 3379, 3435, 3499, 3525
alphw.loc[3525,"texts"]

['(hat wollen als Modalverb)']

In [47]:
alphw.loc[3525,"columns"]

[1, 2]